# TSA Chapter 3: Why ARIMA? Motivation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_motivation/TSA_ch3_motivation.ipynb)

This notebook demonstrates:
- Non-stationary real-world patterns: random walk, drift, deterministic trend
- ADF test comparison showing why ARIMA models are needed.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 300

# Simulate different non-stationary patterns
eps = np.random.normal(0, 1, n)

# Random walk (I(1))
rw = np.cumsum(eps)

# Random walk with drift
rw_drift = np.cumsum(eps + 0.1)

# Deterministic trend + noise
trend = 0.05 * np.arange(n) + np.random.normal(0, 1, n)

# Stationary AR(1)
ar1 = np.zeros(n)
for t in range(1, n):
    ar1[t] = 0.7 * ar1[t-1] + eps[t]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
series = [rw, rw_drift, trend, ar1]
titles = ['Random Walk (I(1))', 'Random Walk with Drift',
          'Deterministic Trend + Noise', 'Stationary AR(1)']
colors = ['#1A3A6E', '#DC3545', '#2E7D32', '#E67E22']

for ax, s, title, c in zip(axes.flatten(), series, titles, colors):
    ax.plot(s, color=c, linewidth=1)
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.set_xlabel('Time')
    adf = adfuller(s)
    label = f'ADF p={adf[1]:.4f}'
    ax.text(0.02, 0.98, label, transform=ax.transAxes, fontsize=9,
            va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.suptitle('Non-Stationary vs Stationary Series', fontweight='bold', fontsize=13)
plt.tight_layout()
save_chart(fig, 'ch3_motivation_nonstationary')
plt.show()

print("Key insight: ARIMA extends ARMA to handle non-stationary (integrated) series")
print("by differencing d times before fitting ARMA(p,q).")